In [ ]:
import pandas as pd
import geopandas as gpd
import boto3
from shapely import wkt 
import time
aws_region = "ap-southeast-2"
schema_name = "fire_data"
table_name ='rfs_fire_data'
result_output_location = "s3://camgoo2-rfs-visualisation/query_results/"
bucket='camgoo2-rfs-visualisation'
athena_client = boto3.client('athena',region_name=aws_region)
s3_client = boto3.client('s3')

In [ ]:
population_df = pd.read_csv('BCP_LGA_for_AUST_short-header/Census BCP Local Government Areas for AUST/AUST/2011Census_B01_AUST_LGA_short.csv')
#keep only nescessary colums
population_df = population_df[['region_id', 'Tot_P_P']]
#Rename the column of the region_id column to be the same as our geo_df dataframe "LGA_CODE11"
population_df = population_df.rename(columns={'region_id':'LGA_CODE11'})
#using .replace method remove 'LGA' from the population_df 'LGA_CODE11' column to be consistent with the 'LGA_CODE11' data in geo_df
#need the regex agrument here otherwise it won't replace
population_df = population_df.replace('LGA','', regex=True)
#save down to csv locally
population_df.to_csv('lga_population/lga_population.csv', header = False, index= False)
#put population_df to s3 bucket
s3 = boto3.client('s3')
#arguments (file to be added, bucket to be added too, name of file in bucket)
s3.upload_file('lga_population/lga_population.csv','camgoo2-rfs-visualisation','lga_population/lga_population.csv')

In [ ]:
geo_df = gpd.read_file('1259030001_lga11aaust_shape/LGA11aAust.shp')
#geospatial polygon needs to be saved down with '|' delimiter as there is "," in geometry and won't all be read
geo_df.to_csv('lga_polygons/lga_polygons.csv', sep = '|', header = False, index= False)
#put geospatial data to bucket
s3 = boto3.client('s3')
#arguments (file to be added, bucket to be added too, name of file in bucket)
s3.upload_file('lga_polygons/lga_polygons.csv','camgoo2-rfs-visualisation','lga_polygons/lga_polygons.csv')

In [ ]:
#JOIN tables USING key fields (lga_code)
query = "SELECT lga_code,lga_name,population,geometry FROM fire_data.lga_polygons JOIN fire_data.lga_population USING (lga_code)"
response = athena_client.start_query_execution(
    QueryString=query,
    ResultConfiguration={"OutputLocation": result_output_location})

#save the query execution id as a variable
execution_id = response["QueryExecutionId"]
time.sleep(5)
response_path = "query_results/"+execution_id+'.csv'
#get the athena csv tat is saved and create a GeoDataFrame 
# create connection to S3 using default config and all buckets within S3
obj = s3_client.get_object(Bucket= bucket, Key= response_path) 
# get object and file (key) from bucket
df = pd.read_csv(obj['Body']) # 'Body' is a key word
# GeoDataFrame
df=gpd.GeoDataFrame(df)
# Well Known Text (WKT) is an Open Geospatial Consortium (OGC) standard that is used to represent spatial data in a textual format
df['geometry']=df['geometry'].apply(wkt.loads)
#save geo data frame down to a GeoJSON file which is required fro our folium chloropeth map
df.to_file('lga_population_geometry/lga_population_geometry.geojson', driver="GeoJSON")